In [25]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2

# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic_clean'
repo_dir = os.path.split(os.getcwd())[0] #.split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

os.chdir(repo_dir)

# # This is needed to add the repo dir to the path so jupyter
# # can load the republic modules directly from the notebooks
# import os
# import sys
# repos = os.getcwd().split('/') #os.path.split(os.getcwd())[0]
# rdir = []
# for p in repos:
#     if p!='':
#         if  p!='notebooks':
#             rdir.append(p)
#         else:
#             break

# repo_dir = '/' + '/'.join(rdir) #os.path.join(rdir)

# if repo_dir not in sys.path:
#     sys.path.append(repo_dir)

# sys.path.append('/Users/rikhoekstra/develop/republic_clean')
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

/Users/rikhoekstra/develop/republic_clean


In [27]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

inv_num = 3760
inv_config = set_config_inventory_num(inv_num,ocr_type="pagexml")

rep_es = initialize_es(host_type="external")

rep_es

In [28]:
import republic.extraction.extract_resolution_metadata as extract_res

# initialise a set of searchers that have all the proposition phrases
# 1. a FuzzyPhraseSearcher that has all the known phrases related to propositions
# 2. a FuzzyTemplateSearcher that has some known proposition opening templates that
#    tries to find phrases that together match the template
# 3. a VariableMatcher that identifies the text between phrase matches in the template
#    and labels these in-between texts as part of the proposer or location.
prop_searchers = extract_res.generate_proposition_searchers()

for searcher_type in prop_searchers:
    print(searcher_type, prop_searchers[searcher_type], '\n')

phrase <fuzzy_search.fuzzy_phrase_searcher.FuzzyPhraseSearcher object at 0x7fac5f65df40> 

template {'proposition_from_correspondence': FuzzyTemplateSearcher(template=FuzzyTemplate(labels={'addressed_to', 'organisation', 'correspondence_from', 'person_location', 'addressee_name', 'proposer_name', 'proposition_opening', 'location', 'title', 'affairs_relation', 'temporal_reference', 'proposition_verb', 'resolution_relation', 'person_role', 'person_name_prefix', 'location_relation', 'residence_relation', 'representation_relation'}, required=['proposition_opening'])), 'proposition_from_statement': FuzzyTemplateSearcher(template=FuzzyTemplate(labels={'addressed_to', 'organisation', 'addressee_name', 'proposer_name', 'proposition_opening', 'location', 'title', 'affairs_relation', 'resident_in', 'temporal_reference', 'proposition_verb', 'resolution_relation', 'person_role', 'person_name_prefix', 'location_relation', 'residence_relation', 'representation_relation'}, required=['proposition_open

In [29]:
# Fetch some resolutions for testing

resolutions = []
for ri, res in enumerate(rep_es.scroll_inventory_resolutions(inv_num)):
    resolutions.append(res)
    if len(resolutions) == 10:
        break

total hits: {'value': 7088, 'relation': 'eq'} 	hits per scroll: 10


In [30]:
import json

# The skip formulas are proposition opening formulas for which we don't know 
# the structure of the full opening sentence, so there is no fitting template.

skip_formulas = {
    'heeft aan haar Hoog Mog. voorgedragen',
    'heeft ter Vergadering gecommuniceert ',
}

for ri, resolution in enumerate(resolutions):
    if len(resolution.evidence) == 0:
        print('resolution without evidence:', resolution.metadata)
    # Fetch all phrase matches for the resolution.
    phrase_matches = extract_res.get_paragraph_phrase_matches(rep_es, resolution)
    # create a copy of the resolution with extended metadata, using the template
    # and variable searchers.
    new_resolution = extract_res.add_resolution_metadata(resolution, phrase_matches,
                                                         prop_searchers['template'],
                                                         prop_searchers['variable'])
    if 'proposition_type' not in new_resolution.metadata or new_resolution.metadata['proposition_type'] is None:
        new_resolution.metadata['proposition_type'] = 'unknown'
    print(json.dumps(new_resolution.metadata, indent=4))


query: {'match': {'text_id.keyword': 'session-1705-07-06-num-1-para-20'}}
{
    "inventory_num": 3760,
    "source_id": "session-1705-07-06-num-1",
    "type": "resolution",
    "id": "session-1705-07-06-num-1-resolution-7",
    "session_date": "1705-07-06",
    "session_id": "session-1705-07-06-num-1",
    "session_num": 1,
    "president": null,
    "session_year": 1705,
    "session_month": 7,
    "session_day": 6,
    "session_weekday": "Lunae",
    "proposition_type": "missive",
    "proposer": {
        "title": {
            "text": "den Secretaris",
            "start_offset": 26,
            "end_offset": 40
        },
        "variable_entity": {
            "text": "Mortaigne ,",
            "start_offset": 40,
            "end_offset": 51
        },
        "full_string": {
            "text": "den Secretasis Mortaigne ",
            "start_offset": 26,
            "end_offset": 51
        }
    },
    "decision": null,
    "resolution_type": "ordinaris",
    "text_page_num

In [31]:
import elasticsearch

In [32]:
elasticsearch.__version__


(7, 16, 0)

In [36]:
def get_phrases(x, rep_es=rep_es):
    phrase_matches = extract_res.get_paragraph_phrase_matches(rep_es, x)
    nr = extract_res.add_resolution_metadata(x, phrase_matches,
                                            prop_searchers['template'],
                                            prop_searchers['variable'])
    return nr

In [33]:
import pandas as pd
sample_sheet = pd.read_excel('/Users/rikhoekstra/Downloads/hoedanigheden-sample-1705-1796.xlsx') # change this to a proper location

In [39]:
r = rep_es.retrieve_resolution_by_id('session-1713-11-08-num-1-resolution-13')

In [55]:
# let's try one resolution. 
# We have to get it from elasticsearch because the spreadsheet with samples does not contain resolution objects

result = get_phrases(r)
# the phrase matcher adds its result to metadata. 
# Perhaps there is a more direct/better way to get the data, but here I get them from the metadata
# but we only want the extra data

keys = [key for key in result.metadata if key not in r.metadata]

query: {'match': {'text_id.keyword': 'session-1713-11-08-num-1-para-24'}}


In [62]:
propresult = {}
for key in keys:
    if key == 'formula':
        for subkey in result.metadata['formula']:
            propresult[subkey] = result.metadata['formula'].get(subkey)['text'] or ''
    else:
        propresult[key] = result.metadata.get(key) or ''
    

In [63]:
propresult

{'proposition_opening': 'IS ter Vergaderinge gelesen de Memorie van ',
 'full_string': 'IS ter Vergaderinge gelesen een nader Memorie ',
 'proposition_origin': 'unidentified',
 'proposition_organisation': 'unidentified',
 'proposer_role': ['extraordinaris', 'Envoyé']}

In [87]:
# now we put all this in one function

def phrase_from_resolution(resolution_id, rep_es=rep_es):
    r = rep_es.retrieve_resolution_by_id(resolution_id)
    result = get_phrases(r)
    keys = [key for key in result.metadata if key not in r.metadata]
    propresult = {'resolution_id': resolution_id}
    propresult['text'] = '\n'.join([p.text for p in r.paragraphs])
    for key in keys:
        if key == 'formula':
            for subkey in result.metadata['formula']:
                propresult[subkey] = result.metadata['formula'].get(subkey)['text'] or ''
        else:
            propresult[key] = result.metadata.get(key) or ''
    return propresult
    

In [88]:
newsheet = sample_sheet.resolution_id.apply(lambda x: phrase_from_resolution(x))

query: {'match': {'text_id.keyword': 'session-1713-11-08-num-1-para-24'}}
query: {'match': {'text_id.keyword': 'session-1727-02-14-num-1-para-13'}}
query: {'match': {'text_id.keyword': 'session-1724-10-02-num-1-para-3'}}
query: {'match': {'text_id.keyword': 'session-1712-02-17-num-1-para-9'}}
query: {'match': {'text_id.keyword': 'session-1708-04-20-num-1-para-32'}}
query: {'match': {'text_id.keyword': 'session-1782-05-17-num-1-para-4'}}
query: {'match': {'text_id.keyword': 'session-1718-06-30-num-1-para-8'}}
query: {'match': {'text_id.keyword': 'session-1792-08-27-num-1-para-7'}}
query: {'match': {'text_id.keyword': 'session-1745-02-15-num-1-para-11'}}
query: {'match': {'text_id.keyword': 'session-1713-06-20-num-1-para-24'}}
query: {'match': {'text_id.keyword': 'session-1781-03-19-num-1-para-19'}}
query: {'match': {'text_id.keyword': 'session-1745-12-13-num-1-para-4'}}
query: {'match': {'text_id.keyword': 'session-1786-05-01-num-1-para-17'}}
query: {'match': {'text_id.keyword': 'sessio

In [89]:
pd.DataFrame().from_records(newsheet).to_clipboard(sep='\t')